In [81]:
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Input, GlobalAveragePooling2D, Dense, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.callbacks import CSVLogger
from keras import backend as K
from keras.constraints import Constraint
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

In [82]:
input_tensor = Input(shape=(224, 224, 3))
base_model = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

C:\Users\nico9\anaconda3\envs\gputest\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [83]:
for layer in base_model.layers:
    layer.trainable=False

x = base_model.output
x = GlobalAveragePooling2D(data_format='channels_last')(x)
x = Dense(7, activation='softmax')(x)

updatedModel = Model(base_model.input, x)
updatedModel.summary()

Model: "model_68"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_76 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_76[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [10]:
train_folder = "../Train"
val_folder = "../Val"

train_datagen = ImageDataGenerator(horizontal_flip=True)
val_datagen = ImageDataGenerator(horizontal_flip=True)

In [11]:
train_batchsize = 32
val_batchsize = 8

train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=train_batchsize,
        class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
        val_folder,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=val_batchsize,
        class_mode="categorical",
        shuffle=False
)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [12]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1 , patience = 10)

csv_logger = CSVLogger('resnet_training.log', separator=',', append=False)
callbacks_list = [reduce_lr, csv_logger]

In [14]:
num_epochs = 200
learning_rate = 1e-4
sgd = SGD(lr=learning_rate, momentum = 0.9)
updatedModel.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['acc'])

In [ ]:
history = updatedModel.fit_generator(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    verbose=1,
    callbacks=callbacks_list
)

model.save('../saves/ResNet50.hdf5')

Epoch 1/200
898/898 [==============================] - 275s 306ms/step - loss: 1.7864 - acc: 0.2784 - val_loss: 0.9016 - val_acc: 0.2179
Epoch 2/200
749/898 [========================>.....] - ETA: 35s - loss: 1.6548 - acc: 0.3504

# Plot


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Fase TEST

In [ ]:
import random
from sklearn import metrics
from sklearn.metrics import classification_report
import seaborn as sns

In [ ]:
def show_confusion_matrix(validations, predictions, labels):

    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=labels,
                yticklabels=labels,
                annot=True,
                fmt='d')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
test_folder = "../Test"
IMAGE_SIZE = 224
random.seed(3)
test_batchsize = 1

In [ ]:
model = load_model('../saves/ResNet50.hdf5')

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
        test_folder,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=test_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
ground_truth = test_generator.classes
label2index = test_generator.class_indices
idx2label = dict((v, k) for k, v in label2index.items())

In [ ]:
predictions = model.predict_generator(test_generator,
                      steps=test_generator.samples / test_generator.batch_size,
                      verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors), test_generator.samples))
labels = ["angry", "disgust", "fear", "happy", "normal", "sad", "surprise"]
show_confusion_matrix(predicted_classes, ground_truth, labels)
print(classification_report(predicted_classes, ground_truth))

# Fase SPARSE

In [ ]:
layers = updatedModel.layers
config = updatedModel.get_config()

In [ ]:
config['layers'][2]['kernel_constraint']=2
config['layers'][7]['kernel_constraint']=2
config['layers'][10]['kernel_constraint']=2
config['layers'][13]['kernel_constraint']=2
config['layers'][14]['kernel_constraint']=2
config['layers'][19]['kernel_constraint']=2
config['layers'][22]['kernel_constraint']=2
config['layers'][25]['kernel_constraint']=2
config['layers'][29]['kernel_constraint']=2
config['layers'][32]['kernel_constraint']=2

config['layers'][35]['kernel_constraint']=2
config['layers'][39]['kernel_constraint']=2
config['layers'][42]['kernel_constraint']=2
config['layers'][45]['kernel_constraint']=2
config['layers'][46]['kernel_constraint']=2
config['layers'][51]['kernel_constraint']=2
config['layers'][54]['kernel_constraint']=2
config['layers'][57]['kernel_constraint']=2
config['layers'][61]['kernel_constraint']=2
config['layers'][64]['kernel_constraint']=2

config['layers'][67]['kernel_constraint']=2
config['layers'][71]['kernel_constraint']=2
config['layers'][74]['kernel_constraint']=2
config['layers'][77]['kernel_constraint']=2
config['layers'][81]['kernel_constraint']=2
config['layers'][84]['kernel_constraint']=2
config['layers'][87]['kernel_constraint']=2
config['layers'][88]['kernel_constraint']=2
config['layers'][93]['kernel_constraint']=2
config['layers'][96]['kernel_constraint']=2

config['layers'][99]['kernel_constraint']=2
config['layers'][103]['kernel_constraint']=2
config['layers'][106]['kernel_constraint']=2
config['layers'][109]['kernel_constraint']=2
config['layers'][113]['kernel_constraint']=2
config['layers'][116]['kernel_constraint']=2
config['layers'][119]['kernel_constraint']=2
config['layers'][123]['kernel_constraint']=2
config['layers'][126]['kernel_constraint']=2
config['layers'][129]['kernel_constraint']=2

config['layers'][133]['kernel_constraint']=2
config['layers'][136]['kernel_constraint']=2
config['layers'][139]['kernel_constraint']=2
config['layers'][143]['kernel_constraint']=2
config['layers'][146]['kernel_constraint']=2
config['layers'][149]['kernel_constraint']=2
config['layers'][150]['kernel_constraint']=2
config['layers'][155]['kernel_constraint']=2
config['layers'][158]['kernel_constraint']=2
config['layers'][161]['kernel_constraint']=2

config['layers'][165]['kernel_constraint']=2
config['layers'][168]['kernel_constraint']=2
config['layers'][171]['kernel_constraint']=2